In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

### Read in simulated profiles

In [ ]:
datapath = "./"
files = sorted([f for f in os.listdir(datapath) if f.endswith('.txt')])

simulated_profiles = {}

for f in files:
    with open(os.path.join(datapath, f), 'r') as file:
        prof_in = pd.read_csv(file,
                              sep = ' ',
                              skiprows = 6,
                              names = ['height_top_(m)', 'height_bottom_(m)', 'height_mid_(m)', 'thickness_(m)', 'temperature_(K)', 'density_(kg/m^3)', 'grain_size_(mm)', 'bond_size_(mm)', 'dd_(-)', 'sp_(-)', 'gt_(swiss_code_F1F2F3)', 'theta_ice_(m^3/m^3)', 'theta_water_(m^3/m^3)', 'theta_air_(m^3/m^3)', 'age_(days)'])
        simulated_profiles[f] = prof_in

### Read in observed profiles

In [ ]:
datapath = "../../download/"
files = sorted([f for f in os.listdir(datapath) if f.endswith('.csv') and f.startswith('BylotDensity')])

observed_profiles1 = {}
observed_profiles2 = {}

for f in files:
    with open(os.path.join(datapath, f), 'r') as file:
        prof_in = pd.read_csv(file,
                              sep = ';',
                              skiprows = 1,
                              usecols=[0, 1, 2, 3],
                              names = ['Date', 'Location', 'Height cm', 'Density kg m-3'])
        observed_profiles1[f] = prof_in[prof_in['Location'].astype(str).str.lower() == 'tundra']
        observed_profiles2[f] = prof_in[prof_in['Location'].astype(str).str.lower() == 'wet 0']

### Make plot

In [ ]:
years_to_plot = [2014, 2015, 2017, 2018, 2019]

# Setup plot
num_cols = 2
num_rows = 3
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 20), sharey=True)
axes = axes.flatten()

# Iterate over the years
for i, year in enumerate(years_to_plot):
    ax = axes[i]

    # Find observed profile
    prof1 = observed_profiles1["BylotDensity" + str(years_to_plot[i]) + ".csv"]
    ax.plot(prof1['Density kg m-3'],
            prof1['Height cm']/100.,
            label='measured - tundra',
            linewidth=2,
            color='black',
            linestyle='-',
            dashes=(2, 2, 4, 2)
           )

    prof2 = observed_profiles2["BylotDensity" + str(years_to_plot[i]) + ".csv"]
    ax.plot(prof2['Density kg m-3'],
            prof2['Height cm']/100.,
            label='measured - wet 0',
            linewidth=2,
            color='black',
            linestyle='-',
            dashes=(4, 4, 2)
           )

    # Find corresponding simulated profiles
    color_map = {
        'default': '#a83232',
        'polar': '#3298a8',
        'redeposit': '#5932a8',
    }
    for f, prof in simulated_profiles.items():
        year = int(f.split('_')[-1].split('-')[0])
        simulation = f.split('_')[-2]
        if year == years_to_plot[i]:
            ax.plot(prof['density_(kg/m^3)'],
                    prof['height_mid_(m)'],
                    label=f'{simulation}',
                    linewidth=2,
                    color=color_map[simulation])
    
    # Set labels and titles, etc
    ax.set_xlabel('Density (kg/m^3)')
    ax.set_ylabel('Height (m)')
    ax.set_title(f'Year {years_to_plot[i]}')
    ax.grid(True)
    ax.legend()
    # Add a), b), c), etc
    ax.text(0.05, 0.95, f'({chr(97+i)})', transform=ax.transAxes, fontsize=14, verticalalignment='top', horizontalalignment='right')

for j in range (i+1, (num_rows * num_cols)):
    ax = axes[j].set_axis_off()
plt.show()